<div style="background-color:#5F9EA0; 
            color:white; 
            padding:15px; 
            border-radius:10px; 
            text-align:center; 
            font-size:30px; 
            font-weight:bold;">
    Detecting Deep Fakes - A Deep Learning Computer Vision Task<br>
    <span style="font-size:20px; font-weight:normal;">
        Hairy Feet: Anna, Jacob, Johannes
    </span>
</div>

<h1 id = setup style = 'font-size:30px; background: linear-gradient(90deg, #5F9EA0, #7AC5CD, #8EE5EE); color: white; padding: 10px;  border-radius: 10px;'>
    1. Introduction: Project Overview & Data 
</h1

<div style="background-color:#D2EFF1;
            color:#333333;
            padding:12px 16px;
            border-radius:10px;
            margin:10px 0;">
   </b>
    
In this project, we build a deep learning computer vision model to distinguish between real human faces, and AI-generated (deepfake) face images. 

We use the **DeepDetect-2025** dataset from Kaggle, which contains over 100k labeled images of faces, split into two classes: real and fake. The goal is to train a binary classifier that can automatically detect whether an image is genuine or AI generated.

# 2. Methods

## 2.1 Importing and Preparing Data

In [2]:
import kagglehub

# Download CIFAKE dataset (for later use)
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

print("Path to dataset files:", path)

# Download Deepfake 20K
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

print("Path to dataset files:", path)

# Download Human-faces dataset
path = kagglehub.dataset_download("/kaggle/input/human-faces-dataset")

# Download 200K dataset
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/deepdetect-2025
Path to dataset files: /kaggle/input/cifake-real-and-ai-generated-synthetic-images
Path to dataset files: /kaggle/input/cifake-real-and-ai-generated-synthetic-images


<div style="background-color:#D2EFF1;
            color:#333333;
            padding:12px 16px;
            border-radius:10px;
            margin:10px 0;">
   </b>

The DeepDetect-2025 dataset is organized into separate "train" and "test" folders, each containing two subfolders:

- "real" = real human face images  
- "fake" = AI-generated (deepfake) face images  

We will:
- Set a fixed image size and batch size
- Point TensorFlow to the "train" and "test" directories


In [2]:
import tensorflow as tf

cifake_train_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train"
cifake_test_dir  = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test"

df20k_dir = "/kaggle/input/deepfake-vs-real-20k/Deep-vs-Real"

hf_dir = "/kaggle/input/human-faces-dataset/Human Faces Dataset"

ds200k_dir = "/kaggle/input/200k-real-vs-ai-visuals-by-mbilal/my_real_vs_ai_dataset/my_real_vs_ai_dataset"



<div style="background-color:#D2EFF1;
            color:#333333;
            padding:12px 16px;
            border-radius:10px;
            margin:10px 0;">
   </b>
    
Now we create the training, validation and test sets. 
We use tf.keras.utils.image_dataset_from_directory to:

- Load images from the "train"  directory
- Automatically split the training data into:
  - **80% training**
  - **20% validation**

The images in the "test" directory are used as a separate held-out test set that we will only use for final evaluation.

In [3]:
# Helper function for getting dataset
def create_ds(directory, subset, labels, IMG_SIZE=(224,224), BATCH_SIZE=None, shuffle=True):
    ds = tf.keras.utils.image_dataset_from_directory(
        directory,
        validation_split=0.2 if subset else None, # Only split if subset is asked for
        subset=subset,
        seed=2025,
        class_names=labels,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        label_mode='int',
        shuffle=shuffle 
    )
    return ds

# --- CIFAKE ---
ds_train_cifake = create_ds(cifake_train_dir, 'training', ['FAKE', 'REAL'])
ds_val_cifake   = create_ds(cifake_train_dir, 'validation', ['FAKE', 'REAL'])

ds_train_cifake = ds_train_cifake.apply(tf.data.experimental.ignore_errors()) # Because it has corrupted files that crash the system otherwise

#-- 200K Dataset--
ds_train_200k = create_ds(ds200k_dir, 'training', ['ai_images', 'real'])
ds_val_200k   = create_ds(ds200k_dir, 'validation', ['ai_images', 'real'])

# CIFAKE hold-out set
ds_test_cifake = create_ds(cifake_test_dir, None, ['FAKE', 'REAL'], shuffle = False)

# Out-of-Distribution (face) sets

# --- Deepfake 20k ---
ds_ood_df20k = create_ds(df20k_dir, None, ['Deepfake', 'Real'], shuffle = False)

# --- HF Dataset ---
ds_ood_hf = create_ds(hf_dir, None , ['AI-Generated Images', 'Real Images'], shuffle = False)

# 2. Concatenate them (Append HF to the end of DF20k)
ds_ood_combined = ds_ood_df20k.concatenate(ds_ood_hf)

print("Datasets created!")

Found 100000 files belonging to 2 classes.
Using 80000 files for training.


I0000 00:00:1765539342.570751      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1765539342.571496      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 100000 files belonging to 2 classes.
Using 20000 files for validation.
Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.
Found 200000 files belonging to 2 classes.
Using 160000 files for training.
Found 200000 files belonging to 2 classes.
Using 40000 files for validation.
Found 20000 files belonging to 2 classes.
Found 19219 files belonging to 2 classes.
Found 9630 files belonging to 2 classes.
Datasets created!


In [5]:
# Batch size
BATCH_SIZE = 64

# Define a helper to enforce shapes
def force_shape(image, label):
    # Explicitly set the shape. This converts RaggedTensors to Dense Tensors.
    image = tf.ensure_shape(image, (224, 224, 3))
    label = tf.ensure_shape(label, ())
    return image, label

# Full dataset function
def ds_full(datasets, weights, batch_size = 64):
    AUTOTUNE = tf.data.AUTOTUNE
    
    ds_full = tf.data.Dataset.sample_from_datasets(
        datasets,
        weights=weights, # Optional: Balance the datasets if one is huge
        stop_on_empty_dataset=False
    )
 
    #ds_full = ds_full.unbatch() # Safety check to stop code from crashing if cell is run twice
    
    ds_full = ds_full.map(force_shape, num_parallel_calls=tf.data.AUTOTUNE)

    ds_full = ds_full.shuffle(buffer_size=1000)
    ds_full = ds_full.batch(batch_size)
    ds_full = ds_full.prefetch(tf.data.AUTOTUNE)

    return ds_full

# Test set function
def prepare_test_set(ds, batch_size=64):
    AUTOTUNE = tf.data.AUTOTUNE

    #ds = ds.unbatch() # Safety check to stop code from crashing if cell is run twice
    
    ds = ds.map(force_shape, num_parallel_calls=AUTOTUNE)
    
    ds = ds.batch(batch_size)
    
    ds = ds.prefetch(AUTOTUNE)
    
    # NOTE: No shuffle(), No sample_from_datasets()
    return ds

# Full training set
ds_train_full = ds_full([ds_train_cifake, ds_train_200k], [1.0, 1.0])
print("Train Dataset Created!")

# Full validation set
ds_val_full = ds_full([ds_val_cifake, ds_val_200k], [1.0, 1.0])
print("Validation Dataset Created!")

# Test set
ds_test_cifake = prepare_test_set(ds_test_cifake)

ds_test_ood = prepare_test_set(ds_ood_combined)
print("Test Datasets Created!")

Train Dataset Created!
Validation Dataset Created!
Test Datasets Created!


## 2.2 Data Exploration and Visualisation

### 2.2.1 Show sample images from the training dataset
We visualize sample images by taking the first batch from the training dataset. Since the dataset is shuffled on loading, each batch is a random collection of images. From this batch, we display the first 9 images in a 3×3 grid along with their corresponding class labels (“real” or “fake”).

We do this to perform an initial quality check of the dataset: visual inspection allows us to confirm that the images were loaded correctly, that the labels correspond to the expected classes, and that there are no obvious issues such as corrupted files, incorrect preprocessing, or mislabeled images. Showing random samples also helps us get an intuitive understanding of what the model will see during training and whether the dataset contains sufficient visual variability for effective learning.

### 2.2.2 Check class distribution
We assess the class distribution by counting how many images belong to each category (“real” and “fake”). A balanced dataset is important because severe class imbalance can bias the model toward predicting the majority class. By examining the distribution visually and numerically, we ensure that the model will be trained on approximately equal amounts of real and AI-generated images, reducing the risk of skewed learning or misclassification patterns.

### 2.2.3 Inspect Image Shape and Label Format
We inspect the shape of one batch to verify that the images and labels are formatted as expected. The image batch typically has the shape (batch_size, height, width, channels) e.g., (32, 224, 224, 3), indicating 32 RGB images of size 224×224 pixels.

The label batch should have the shape (batch_size,), containing one integer label per image. Confirming these shapes helps ensure that the data pipeline is correctly configured before building the neural network model.

The batch shape (32, 224, 224, 3) confirms that images are correctly loaded in batches of 32, each resized to 224×224 pixels with three RGB channels. The label batch (32,) shows that each image has one corresponding class label. The example label 0/1 indicates that the labeling system is functioning as expected and correctly maps images to their respective classes. Overall, the data pipeline is properly structured for model training.

## 2.3 Data Preprocessing

### 2.3.1 Data Augmentation
We apply data augmentation to artificially increase the variability of the training dataset and improve the model’s ability to generalize. The augmentation pipeline randomly flips, rotates, and zooms images during training, introducing meaningful variations that help the model become more robust to common transformations. This reduces overfitting and improves performance on unseen data.


### 2.3.2 Visualizing Data Augmentation
We visualize a batch of augmented images to verify that the applied transformations (flipping, rotation, and zoom) behave as intended and produce realistic variations. This quick inspection ensures that augmentation does not distort the data in a harmful way and confirms that the model will receive meaningful, correctly transformed inputs during training.

## 2.4 Neural Models

### 2.4.1 Custom CNN

**Architecture**

In [6]:
from tensorflow import keras
from tensorflow.keras import layers

# Custom-made CNN classifier 
custom_cnn = keras.Sequential([
    # Input layer
    layers.Input(shape=(224, 224, 3)),
    
    # First convolution + maxpool block (32 filters)
    layers.Conv2D(filters=32, kernel_size=3, padding='same'),
    layers.BatchNormalization(), 
    layers.Activation('relu'),   
    layers.MaxPool2D(),

    # Second convolution + maxpool block (64 filters)
    layers.Conv2D(filters=64, kernel_size=3, padding='same'),
    layers.BatchNormalization(), 
    layers.Activation('relu'),   
    layers.MaxPool2D(),

    # Third convolution + maxpool block (128 filters)
    layers.Conv2D(filters=128, kernel_size=3, padding='same'),
    layers.BatchNormalization(), 
    layers.Activation('relu'),   
    layers.MaxPool2D(),

    #Fourth convolution + maxpool block (256 filters)
    layers.Conv2D(filters=256, kernel_size=3, padding='same'),
    layers.BatchNormalization(), 
    layers.Activation('relu'),
    # removed maxpool to preserve fine-grained details

    # Global average pooling block
    layers.GlobalAveragePooling2D(),

    # Fully connected block
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64,activation = 'relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation = 'relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid'),
    
])

custom_cnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # Slower, more precise training
    loss='binary_crossentropy',
    metrics=['accuracy']
)

custom_cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 433,601 (1.65 MB)

 Trainable params: 432,641 (1.65 MB)

 Non-trainable params: 960 (3.75 KB)

### 2.4.2 Pre-trained CNN

**Architecture**



In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

IMG_SIZE = 224
BATCH_SIZE = 32

# Load pretrained EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False  # freeze weights for transfer learning

# Build model
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = tf.keras.applications.efficientnet.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_cnn = models.Model(inputs, outputs)

model_cnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_cnn.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

### 2.4.3 Visual Transformer

**Architecture**

In [8]:
# ------------------------------
# 1️⃣ Imports & Setup
# ------------------------------
import torch
from torch import nn
from torch.optim import AdamW
from transformers import ViTForImageClassification
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm  # Keras-style progress bar
import copy
import os

# Enable faster TF GPU memory allocation
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

class ViTClassifier:
    def __init__(self, model_name="google/vit-base-patch16-224", num_labels=2, learning_rate=2e-5):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🔧 Initializing ViTClassifier on {self.device}...")

        # 1. Load Model
        self.model = ViTForImageClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            ignore_mismatched_sizes=True
        ).to(self.device)

        # ---------------------------------------------------------
        # ❄️ FREEZING 
        # ---------------------------------------------------------
        # Loop through the base model (self.model.vit) and turn off gradients
        for param in self.model.vit.parameters():
            param.requires_grad = False
            
        # Ensure the Classifier Head (self.model.classifier) is OPEN for training
        for param in self.model.classifier.parameters():
            param.requires_grad = True
        # ---------------------------------------------------------

        # 2. Compile (Optional Speedup)
        try:
            self.model = torch.compile(self.model)
        except:
            pass

        # 3. Optimization (Only optimize parameters that require gradients)
        # We filter the parameters to ensure the optimizer doesn't track frozen ones
        trainable_params = [p for p in self.model.parameters() if p.requires_grad]
        self.optimizer = AdamW(trainable_params, lr=learning_rate)
        
        self.criterion = nn.CrossEntropyLoss()
        self.scaler = torch.amp.GradScaler('cuda')

        self.history = {
            'loss': [], 'accuracy': [],
            'val_loss': [], 'val_accuracy': []
        }

    def _tf_to_torch_fast(self, images, labels):
        """
        Internal helper: Optimized data transfer from TF (CPU) -> PyTorch (GPU).
        Expects images to be uint8 [0, 255].
        """
        # Zero-copy transfer to CPU tensor
        images = torch.from_numpy(np.array(images)) 
        labels = torch.from_numpy(np.array(labels)).long().to(self.device)

        # Move to GPU (non_blocking allows async transfer)
        images = images.to(self.device, non_blocking=True)

        # Permute (Batch, Height, Width, Channel) -> (Batch, Channel, Height, Width)
        # And Normalize [0, 255] -> [0.0, 1.0]
        images = images.permute(0, 3, 1, 2).float() / 255.0
        
        return images, labels

    def _validate(self, val_loader):
        """Internal helper for validation loop"""
        self.model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = self._tf_to_torch_fast(images, labels)
                
                with torch.amp.autocast('cuda'):
                    outputs = self.model(images)
                    loss = self.criterion(outputs.logits, labels)
                
                val_loss += loss.item() * labels.size(0)
                preds = torch.argmax(outputs.logits, dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        # Avoid division by zero
        if total == 0: return 0.0, 0.0
        return val_loss / total, correct / total

    def train(self, train_ds, val_ds, epochs=10, patience=3):
        """
        Main training loop with:
         - TQDM Progress Bar (Keras style)
         - Early Stopping
         - History tracking
        """
        # Convert TF datasets to numpy iterables
        # We try to get the length for the progress bar if possible
        try:
            n_batches = len(train_ds)
        except:
            n_batches = None

        train_loader = tfds.as_numpy(train_ds)
        val_loader = tfds.as_numpy(val_ds)
        
        best_val_loss = float('inf')
        patience_counter = 0
        best_model_weights = None
        
        print(f"\n🚀 Starting Training (Max Epochs: {epochs}, Patience: {patience})")

        for epoch in range(epochs):
            self.model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            
            # --- Progress Bar ---
            with tqdm(train_loader, total=n_batches, unit="batch", leave=True) as pbar:
                pbar.set_description(f"Epoch {epoch+1}/{epochs}")
                
                for batch_idx, (images, labels) in enumerate(pbar):
                    images, labels = self._tf_to_torch_fast(images, labels)
                    
                    self.optimizer.zero_grad()
                    
                    # Mixed Precision Forward
                    with torch.amp.autocast('cuda'):
                        outputs = self.model(images)
                        loss = self.criterion(outputs.logits, labels)
                    
                    # Mixed Precision Backward
                    self.scaler.scale(loss).backward()
                    self.scaler.step(self.optimizer)
                    self.scaler.update()

                    # Metrics
                    batch_size = labels.size(0)
                    running_loss += loss.item() * batch_size
                    
                    preds = torch.argmax(outputs.logits, dim=1)
                    batch_correct = (preds == labels).sum().item()
                    correct += batch_correct
                    total += batch_size
                    
                    # Live Update of Bar
                    current_acc = correct / total
                    current_loss = running_loss / total
                    pbar.set_postfix({"loss": f"{current_loss:.4f}", "acc": f"{current_acc:.4f}"})

            # --- Validation & History ---
            val_loss, val_acc = self._validate(val_loader)
            
            self.history['loss'].append(current_loss)
            self.history['accuracy'].append(current_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_accuracy'].append(val_acc)
            
            print(f"    Validation - loss: {val_loss:.4f} - acc: {val_acc:.4f}")

            # --- Early Stopping ---
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_model_weights = copy.deepcopy(self.model.state_dict())
                # print("    ⭐ Saved Best Model")
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"\n🛑 Early stopping triggered! Restoring best weights.")
                    self.model.load_state_dict(best_model_weights)
                    break
        
        # Restore best weights if finished naturally
        if best_model_weights is not None and patience_counter < patience:
             self.model.load_state_dict(best_model_weights)

    def plot_history(self):
        """Plots accuracy and loss graphs similar to Keras."""
        if not self.history['loss']:
            print("No training history to plot.")
            return

        epochs_range = range(1, len(self.history['loss']) + 1)
        
        plt.figure(figsize=(12, 5))

        # Plot Loss
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, self.history['loss'], 'bo-', label='Training Loss')
        plt.plot(epochs_range, self.history['val_loss'], 'r-', label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        # Plot Accuracy
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, self.history['accuracy'], 'bo-', label='Training Acc')
        plt.plot(epochs_range, self.history['val_accuracy'], 'r-', label='Validation Acc')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.show()

    def predict(self, tf_dataset):
        """Runs inference on a dataset and returns (y_true, y_pred_prob)"""
        loader = tfds.as_numpy(tf_dataset)
        self.model.eval()
        
        all_preds = []
        all_labels = []
        
        print(f"🔍 Running Prediction...")
        with torch.no_grad():
            for images, labels in tqdm(loader, unit="batch"):
                images, labels = self._tf_to_torch_fast(images, labels)
                
                with torch.amp.autocast('cuda'):
                    outputs = self.model(images)
                    
                # Get probabilities (softmax)
                probs = torch.softmax(outputs.logits, dim=1)
                
                all_preds.append(probs.cpu().numpy())
                all_labels.append(labels.cpu().numpy())
                
        return np.concatenate(all_labels), np.concatenate(all_preds)

    def save(self, path="vit_model.pth"):
        torch.save(self.model.state_dict(), path)
        print(f"💾 Model saved to: {path}")

    def load(self, path="vit_model.pth"):
        self.model.load_state_dict(torch.load(path))
        self.model.eval()
        print(f"📂 Model loaded from: {path}")

## 2.5 Loss and Performance Metrics

## 2.6 Fitting


### 2.6.1 Early Stopping for keras models

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',        # Watch validation loss (metric of truth)
    patience=3,                # Wait 3 epochs before stopping (gives it a chance to recover)
    restore_best_weights=True, # CRITICAL: Go back to the best weights, not the last ones
    verbose=1                  # Print a message when it triggers
)

### Function for training, predicting and cleaning (llm) for keras models

In [13]:
import gc
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt  # Required to render plots inside a function

# Initialize the global results dictionary if it doesn't exist
if 'all_results' not in globals():
    all_results = {}

# Define your test sets map
test_sets = {
    "CIFAKE_Test": ds_test_cifake,
    "OOD_Test":    ds_test_ood
}

def train_eval_clean(model, model_name, train_ds, val_ds, test_sets, epochs=30):
    """
    1. Trains the model.
    2. PLOTS the history (New Step).
    3. Evaluates on all test/OOD sets.
    4. Saves results to global 'all_results'.
    5. Saves model to disk.
    6. Deletes model and clears VRAM.
    """
    print(f"\n{'='*60}")
    print(f"🚀 STARTING PIPELINE FOR: {model_name}")
    print(f"{'='*60}")

    # ---------------------------------------------------------
    # 1. TRAIN
    # ---------------------------------------------------------
    print(f"📉 Training {model_name}...")
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        verbose=1,
        callbacks=[early_stopping] # Assumes 'early_stopping' is defined globally
    )

    # ---------------------------------------------------------
    # 1.5. PLOT HISTORY (Added)
    # ---------------------------------------------------------
    print(f"📊 Plotting training history for {model_name}...")
    
    # Convert history to pandas DataFrame
    history_frame = pd.DataFrame(history.history)
    
    # Plot Loss
    # We use plt.show() to force the plot to render immediately during function execution
    history_frame.loc[:, ['loss', 'val_loss']].plot(title=f"{model_name} - Loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show() 
    
    # Plot Accuracy (Checks if 'binary_accuracy' exists, falls back to 'accuracy' if needed)
    acc_key = 'binary_accuracy' if 'binary_accuracy' in history_frame.columns else 'accuracy'
    val_acc_key = f'val_{acc_key}'
    
    if acc_key in history_frame.columns:
        history_frame.loc[:, [acc_key, val_acc_key]].plot(title=f"{model_name} - Accuracy")
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.show()

    # ---------------------------------------------------------
    # 2. EVALUATE
    # ---------------------------------------------------------
    print(f"\n🔍 Evaluating {model_name} on {len(test_sets)} datasets...")
    
    # Create entry in global results dict
    all_results[model_name] = {}

    for ds_name, ds in test_sets.items():
        print(f"   • Predicting on {ds_name}...")
        
        # A. Predict
        preds = model.predict(ds, verbose=0) # verbose=0 to keep logs clean
        
        # Handle shapes: (N, 2) -> (N,) or (N, 1) -> (N,)
        if preds.shape[-1] > 1:
            y_pred = preds[:, 1] 
        else:
            y_pred = preds.flatten()
            
        # B. Get True Labels
        y_true = np.concatenate([y for x, y in ds], axis=0)
        
        # C. Store
        all_results[model_name][ds_name] = {
            'y_pred': y_pred,
            'y_true': y_true
        }

    # ---------------------------------------------------------
    # 3. SAVE TO DISK
    # ---------------------------------------------------------
    filename = f"{model_name}_final.keras"
    model.save(filename)
    print(f"\n💾 Model saved to: {filename}")

    # ---------------------------------------------------------
    # 4. CLEANUP (The most important part)
    # ---------------------------------------------------------
    print(f"🧹 Scrubbing VRAM...")
    del model
    tf.keras.backend.clear_session()
    gc.collect()
    print(f"✨ {model_name} pipeline complete. GPU is ready for next model.\n")
    
    return history

### 2.6.2 Custom CNN

In [ ]:
hist_custom = train_eval_clean(
    custom_cnn, 
    "Custom_CNN", 
    ds_train_full, 
    ds_val_full, 
    test_sets,
    epochs=30 # Change to 30 for real run
)


🚀 STARTING PIPELINE FOR: Custom_CNN
📉 Training Custom_CNN...
Epoch 1/30


I0000 00:00:1765540287.105335     100 service.cc:148] XLA service 0x7ad04c413dc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765540287.110854     100 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765540287.110874     100 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765540287.868625     100 cuda_dnn.cc:529] Loaded cuDNN version 90300


      1/Unknown 19s 19s/step - accuracy: 0.5469 - loss: 0.7118

I0000 00:00:1765540299.472718     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   3750/Unknown 574s 148ms/step - accuracy: 0.6703 - loss: 0.5836

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


3750/3750 ━━━━━━━━━━━━━━━━━━━━ 724s 188ms/step - accuracy: 0.6703 - loss: 0.5836 - val_accuracy: 0.6483 - val_loss: 0.6354
Epoch 2/30
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 489s 130ms/step - accuracy: 0.7560 - loss: 0.4740 - val_accuracy: 0.7082 - val_loss: 0.5726
Epoch 3/30
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 481s 128ms/step - accuracy: 0.7837 - loss: 0.4315 - val_accuracy: 0.7090 - val_loss: 0.5621
Epoch 4/30
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 491s 131ms/step - accuracy: 0.8034 - loss: 0.3997 - val_accuracy: 0.6987 - val_loss: 0.6368
Epoch 5/30
1506/3750 ━━━━━━━━━━━━━━━━━━━━ 4:18 115ms/step - accuracy: 0.8162 - loss: 0.3804

### 2.6.3 Pre-trained CNN

In [18]:
hist_cnn = train_eval_clean(
    model_cnn, 
    "Pretrained_CNN", 
    ds_train_full, 
    ds_val_full, 
    test_sets,
    epochs=30 # Change to 30 for real run
)



🚀 STARTING PIPELINE FOR: Pretrained_CNN
📉 Training Pretrained_CNN...
Epoch 1/2
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 385s 96ms/step - accuracy: 0.7314 - loss: 0.5168 - val_accuracy: 0.7561 - val_loss: 0.4935
Epoch 2/2
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 349s 93ms/step - accuracy: 0.7620 - loss: 0.4738 - val_accuracy: 0.7595 - val_loss: 0.4850
Restoring model weights from the end of the best epoch: 2.

🔍 Evaluating Pretrained_CNN on 2 datasets...
   • Predicting on CIFAKE_Test...
   • Predicting on OOD_Test...


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG



💾 Model saved to: Pretrained_CNN_final.keras
🧹 Scrubbing VRAM...
✨ Pretrained_CNN pipeline complete. GPU is ready for next model.



### 2.6.3 Visual Transformer

In [25]:
# 1. Initialize
vit_model = ViTClassifier(num_labels=2)

# 2. Train (Pass val set and patience)
# Note: Ensure datasets are Unbatched in TF or handle batching inside appropriately.
# Since my class assumes tfds.as_numpy iterates batches, ensure your input DS is BATCHED.
vit_model.train(ds_train_full, ds_val_full, epochs=20, patience=3)

# 3. Plot
vit_model.plot_history()

# Predictions on test sets
# 1. Initialize the entry for this specific model
model_name = "ViT_Base_Frozen"
all_results[model_name] = {}

print(f"🚀 Starting Evaluation for {model_name}...")

# 2. Iterate through your existing test_sets map
for ds_name, ds in test_sets.items():
    print(f"   • Predicting on {ds_name}...")
    
    # Run inference using the class method
    # This returns (True Labels, Predicted Probabilities [N, 2])
    y_true, y_probs = vit_model.predict(ds)
    
    # We want the probability of the "Positive" class (Index 1)
    # y_probs is shape [N, 2], so we take all rows, column 1
    y_pred = y_probs[:, 1]
    
    # 3. Store in the global dictionary
    all_results[model_name][ds_name] = {
        'y_pred': y_pred,
        'y_true': y_true
    }

print(f"\n✅ Results for {model_name} saved to 'all_results'.")

# Saving the model
# Save with a specific filename
vit_model.save("ViT_Frozen_Base_Final.pth")

🚀 Starting Evaluation for ViT_Base_Frozen...
   • Predicting on CIFAKE_Test...
🔍 Running Prediction...


  0%|          | 0/313 [00:00<?, ?batch/s]

   • Predicting on OOD_Test...
🔍 Running Prediction...


  0%|          | 0/451 [00:00<?, ?batch/s]

Invalid SOS parameters for sequential JPEG



✅ Results for ViT_Base_Frozen saved to 'all_results'.


# 3. Results

### Note for Johannes/Anna.

the results for all the models can be accessed through the dictionary `all_results`. If you want to load in trained model then use the `tf.keras.models.load_model("model_name")` for the custom CNN and the pre-trained cnn. This way you can make more predictions with the models. 

## 3.1 Hold-out set performance 

In [31]:
# the hold-out set is the ds_test_cifake


KeyError: 'cifake'

## 3.2 Out-of-distribution set performance

In [ ]:
# the ood set is the ds_test_ood

## 3.3 Performance on custom images 

In [ ]:
# use a pic of johannes and a pic of ai generated johannes to see if it predicts correctly 

# 4. Conclusion/Discussion

possible discussion points:
> fine tune the base layers in both pre-trained classifier and visual transformer rather than just the classifier head (time/memory constraints).


> Modify the custom cnn to see how simple it can be made while still maintaining good performance. 

# 5. References

# 6. Division of Labour